# RDDs

## Download and install Spark

In [1]:
!ls

sample_data


In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubu

## Setup environment

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading Chicago's police station dataset

In [4]:
!wget -O police-stations.csv https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
!ls -l

--2019-12-29 14:15:51--  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘police-stations.csv’

police-stations.csv     [ <=>                ]   5.57K  --.-KB/s    in 0s      

2019-12-29 14:15:51 (576 MB/s) - ‘police-stations.csv’ saved [5699]

total 220612
-rw-r--r--  1 root root      5699 Aug 19 01:26 police-stations.csv
drwxr-xr-x  1 root root      4096 Dec 18 16:52 sample_data
drwxrwxr-x 13 1000 1000      4096 Jun  1  2018 spark-2.3.1-bin-hadoop2.7
-rw-r--r--  1 root root 225883783 Jun  1  2018 spark-2.3.1-bin-hadoop2.7.tgz


## RDDs setup

In [0]:
ps= sc.textFile("police-stations.csv")

In [6]:
ps.first()

'DISTRICT,DISTRICT NAME,ADDRESS,CITY,STATE,ZIP,WEBSITE,PHONE,FAX,TTY,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION'

In [0]:
ps_header= ps.first()

In [0]:
ps_rest=ps.filter(lambda line: line!=ps_header)

In [9]:
ps_rest.first()

'1,Central,1718 S State St,Chicago,IL,60616,http://home.chicagopolice.org/community/districts/1st-district-central/,312-745-4290,312-745-3694,312-745-3693,1176569.052,1891771.704,41.85837259,-87.62735617,"(41.8583725929, -87.627356171)"'

In [10]:
ps_rest.map(lambda line: line.split(",")).collect()

[['1',
  'Central',
  '1718 S State St',
  'Chicago',
  'IL',
  '60616',
  'http://home.chicagopolice.org/community/districts/1st-district-central/',
  '312-745-4290',
  '312-745-3694',
  '312-745-3693',
  '1176569.052',
  '1891771.704',
  '41.85837259',
  '-87.62735617',
  '"(41.8583725929',
  ' -87.627356171)"'],
 ['6',
  'Gresham',
  '7808 S Halsted St',
  'Chicago',
  'IL',
  '60620',
  'http://home.chicagopolice.org/community/districts/6th-district-gresham/',
  '312-745-3617',
  '312-745-3649',
  '312-745-3639',
  '1172283.013',
  '1853022.646',
  '41.75213684',
  '-87.64422891',
  '"(41.7521368378',
  ' -87.6442289066)"'],
 ['11',
  'Harrison',
  '3151 W Harrison St',
  'Chicago',
  'IL',
  '60612',
  'http://home.chicagopolice.org/community/districts/11th-district-harrison/',
  '312-746-8386',
  '312-746-4281',
  '312-746-5151',
  '1155244.069',
  '1897148.755',
  '41.87358229',
  '-87.70548813',
  '"(41.8735822883',
  ' -87.705488126)"'],
 ['16',
  'Jefferson Park',
  '5151 N M

**How many police stations are there?**

In [12]:
  ps_rest.map(lambda line: line.split(",")).count()

24

**Display the District ID, District name, Address and Zip for the police station with District ID 7**



In [15]:
(ps_rest.filter(lambda line: line.split(",")[0]=='7')).map( lambda line:( line.split(',')[0],line.split(',')[1],line.split(',')[2],line.split(',')[5]) ).collect()

[('7', 'Englewood', '1438 W 63rd St', '60636')]

**Police stations 10 and 11 are geographically close to each other. Display the District ID, District name, address and zip code**

In [16]:
(ps_rest.filter(lambda line: line.split(",")[0] in ['10','11'])).map( lambda line:( line.split(',')[0],line.split(',')[1],line.split(',')[2],line.split(',')[5]) ).collect()

[('11', 'Harrison', '3151 W Harrison St', '60612'),
 ('10', 'Ogden', '3315 W Ogden Ave', '60623')]